In [ ]:
import glob
import h5py
import os

import cmasher as cmr
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point

In [ ]:
PATH_DATA = os.path.join('..', 'data')
PATH_DATA_SOURCE = os.path.join(PATH_DATA, 'raw_data')
PATH_PROCESSED = os.path.join(PATH_DATA, "processed_data")
PATH_SHAPEFILE = os.path.join(PATH_DATA_SOURCE, 'geodata', 'AUT_adm0.shp')

FOLDERS = [folder for folder in os.listdir(PATH_PROCESSED) if folder.startswith('dataparquet')]
FOLDERS.sort(reverse=True)

DIR_DATABUNDLE = FOLDERS[0]

PATH_OUTPUT = os.path.join(PATH_DATA, "hail_days", DIR_DATABUNDLE)

QUALITY_HATCH = False
DATA_QUALITY_THRESHOLD = 0.4

In [ ]:
df_full = pd.read_parquet(os.path.join(PATH_PROCESSED, DIR_DATABUNDLE))

df_full["lon"] /= 1000000
df_full["lat"] /= 1000000

In [ ]:
target_cols = [col for col in df_full.columns if col.startswith('data_mehs')]

In [ ]:
if QUALITY_HATCH:
    quality_reports = glob.glob(os.path.join(PATH_DATA_SOURCE, 'ZAMG', 'dataquality_map', '2023_12_04','*.h5'))
    quality_file = h5py.File(quality_reports[0])

    data_lat = quality_file["atnt_grid_lat"][:]
    data_lon = quality_file["atnt_grid_lon"][:]
    data_quality = quality_file['data'][:]

    data_xr_quality = xr.DataArray(
                            data_quality,
                            coords=dict(
                                lon=(["y", "x"], data_lon),
                                lat=(["y", "x"], data_lat),
                            ),
                            dims=["y", "x"],
                            name = 'quality'
                        )

    data_pandas_quality = data_xr_quality.to_dataframe()
    data_schraffur_quality = data_pandas_quality[(data_pandas_quality['quality'].isna()) | (data_pandas_quality['quality'] < DATA_QUALITY_THRESHOLD)]

In [ ]:
if QUALITY_HATCH:
    elevation_reports = glob.glob(os.path.join(PATH_DATA_SOURCE, 'ZAMG', 'dataquality_map', '2023_11_20','*.h5'))
    elevation_file = h5py.File(elevation_reports[0])

    data_lat_elevation = elevation_file["atnt_grid_lat"][:]
    data_lon_elevation = elevation_file["atnt_grid_lon"][:]
    data_elevation = elevation_file['data'][:]

    data_xr_elevation = xr.DataArray(
                            data_elevation,
                            coords=dict(
                                lon=(["y", "x"], data_lon_elevation),
                                lat=(["y", "x"], data_lat_elevation),
                            ),
                            dims=["y", "x"],
                            name = 'elevation'
    )
    
    data_pandas_elevation = data_xr_elevation.to_dataframe()
    data_schraffur_elevation = data_pandas_elevation[data_pandas_elevation['elevation'].isna()]

In [ ]:
austria = gpd.read_file(PATH_SHAPEFILE)

In [ ]:
os.makedirs(PATH_OUTPUT, exist_ok=True)

cmap = cmr.get_sub_cmap('plasma', 0.05, 0.9)
vmin = 1

for col in target_cols:        
    print(f"Computing plots for column {col}", flush=True)
        
    ## Prepare dataframe
    df_speccolumn = df_full[["lon", "lat", col]].copy()

    df_speccolumn.loc[:, "sees_hail"] = df_speccolumn[col] > 0

    df_agg = df_speccolumn.groupby(['lon', 'lat']).agg({"sees_hail" : ['sum']}).reset_index()
    df_agg.columns = [c[0] for c in list(df_agg.columns)[:2]] + ['hail_events']
    
    for clip_to_austria in [False]:
        print(f"Clipping to Austria: {clip_to_austria}", flush=True)
        
        if clip_to_austria:
            geometry = [Point(xy) for xy in zip(df_agg['lon'], df_agg['lat'])]
            gdf = gpd.GeoDataFrame(df_agg, geometry=geometry, crs="EPSG:4326")
            df_agg = gpd.sjoin(gdf, austria, how="inner", predicate='within')[df_agg.columns]
        
        infix_clipped = "_clipped_AUT" if clip_to_austria else ""

        filename = f"hailriskat_{col}{infix_clipped}"
        
        df_agg.to_csv(os.path.join(PATH_OUTPUT, f"{filename}.csv"), index=False)

        vmax = df_agg['hail_events'].max()
        
        ## Plot
        plt.rcParams.update({'font.size': 18})
        ax = plt.figure(figsize=(15, 10))
    
        m = Basemap(projection='lcc', resolution='f', lat_0=47.7, lon_0=13.3, width=6.0E5, height=3.35E5)
        m.drawmapboundary()
        m.drawcountries(linewidth=2)
    
        m.scatter(df_agg['lon'], df_agg['lat'], c=df_agg['hail_events'], cmap=cmap, s=2, latlon=True, vmin=vmin, vmax=vmax)
        #plt.title(col)
    
        plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}.png"), bbox_inches="tight")
        plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}.pdf"), bbox_inches="tight")
    
        if QUALITY_HATCH:
            m.scatter(data_schraffur_quality['lon'], data_schraffur_quality['lat'], s=0.01, edgecolor='white', linewidth=3, latlon=True, facecolor='black', hatch='x')
            m.scatter(data_schraffur_elevation['lon'], data_schraffur_elevation['lat'], s=0.01, edgecolor='white', linewidth=3, latlon=True, facecolor='black', hatch='x')
            plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}_hatched.png"), bbox_inches="tight")
            plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}_hatched.pdf"), bbox_inches="tight")
    
        plt.close()

        plt.rcParams.update({'font.size': 30})
        fig = plt.figure(figsize=(3, 8))
        ax1 = fig.add_axes([0.05, 0.80, 0.2, 0.9])

        cb1 = mpl.colorbar.ColorbarBase(
                                            ax1,
                                            cmap=cmap,
                                            norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax),
                                            orientation='vertical'
        )

        cb1.set_label('hail days')
        
        plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}_cmap.png"), bbox_inches="tight")
        plt.savefig(os.path.join(PATH_OUTPUT, f"{filename}_cmap.pdf"), bbox_inches="tight")

        plt.close()

print("Fin.", flush=True)